## Preprocessing the Dataset

#### Notebook Description
This is a notebook that preprocesses the dataset for the project. It is designed to be run in Google Colab,  but can be run locally. If run locally, the dataset all_hourly_data.h5 should be in a folder "/data" within the project folder. The output of the processing step are the csv file which contains the processed and synthetic data, and a series of numpy arrays for use by the Mimic3Dataset class.

#### MIMIC-III Dataset
The MIMIC-III dataset is a large dataset of de-identified health data. It is available from the Physionet website: https://physionet.org/content/mimiciii/1.4/. It contains health related data from patients who were admitted to the critical care units of Beth Isreal Deaconess Medical Center from 2001 to 2012. It consists of both static and dynamic data. In addition to patient demographics, there is data acquired from the hospital that is not necessarily related to the patient's intensive care unit (ICU) stay, and the data related to the ICU stay such as vital signs, laboratory tests, diagnostic codes, medications, etc. This includes data from bedside monitoring systems, and the "chart" (electronic health record) data including provider's notes.
Patient Characteristics:
There are data associated with 53,423 distinct adult (age 16 and above) hospital admissions of 38,597 distinct adult patients. There are also data for 7870 neonate admissions, which we will eliminate from our data for analysis.

#### MIMIC-Extract data
MIMIC Extract is an open source pipeline to "extract, preprocess, and represent data from MIMIC-III v1.4.", as described in the paper "MIMIC-Extract: A Data Extraction, Preprocessing, and Representation Pipeline for MIMIC-III." (arXiv:1907.08322). The code is available at: [MIMIC_Extract](https://github.com/MLforHealth/MIMIC_Extract).
The extraction process results in data that includes well-formatted time-series data for clinically-meaningful prediction tasks. The time-varying data are discretized into hourly buckets, standardized, and aggregated into clinically meaningful representations.
The cohort of patients includes all adult patients whose ICU stay is their first, and that stay lasts at least 12 hours, and less that 10 days. This represents a generic cohort, not task-specific, and therefore can be adapted for many different clinical prediction tasks.
The extraction procedure is extensible and you may configure the cohort selection process. Using the default parameters for cohort selection, the data consists of 34,472 patients.About the all_hourly_data.h5 file:  It is a large file that contains all the hourly data for all patients in the MIMIC-III database. The "all_hourly_data.h5" file is the result of the pipeline using the default parameters, and is supplied by the authors of the MIMIC-Extract paper. The file is available for download from Google Cloud (with appropriate credentialing from [Physionet](https://mimic.mit.edu/docs/gettingstarted/)), and a link is provided on the MIMIC_Extract GitHub page.

The h5 datafile format is a hierarchical data format that is optimized for storing large amounts of data. It is a binary format, and is not human-readable. The data is stored in a hierarchical filesytem-like format, with each node in the hierarchy being a group or a dataset. A group is a container that can hold datasets and other groups. A dataset is a multidimensional array of data elements. The data can be read directly into a pandas dataframes, which is what we will do in this notebook.

The following three cells are used to mount the Google Drive to the Colab notebook. This is only necessary if running in Colab. Skip to the next cell if running locally.

In [ ]:
# Where are we?
%ls

sample_data/


### Mount Google Drive
I am setting this project up to run in Google Colab, and I am using Google Drive to store the project code and data. The following cell mounts the Google Drive to the Colab notebook.

In [ ]:
# Mount our Google Drive
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# Change to the project directory - yours may be different
%cd /content/drive/MyDrive/dl4h_project/DynST/
%ls

/content/drive/MyDrive/dl4h_project/DynST
causal.ipynb            outputs/                          pyproject.toml
config.yaml             poetry.lock                       README.md
coxph_model.ipynb       project_causal.ipynb              results20230416.txt
data/                   project_coxph_model.ipynb         run.py
dl4hProjectSetup.ipynb  project_model.ipynb               src/
multirun/               project_preprocess_dataset.ipynb


In [ ]:
import logging

from pathlib import Path
import numpy as np
import pandas as pd
from scipy.special import expit, logit

In [ ]:
log = logging.getLogger(__name__)

### Mimic3Pipeline Class
This class contains functions which prepare the data for analysis. The source of the data is the h5 file that is the output of the MIMIC-Extract pipeline. The data is read into a pandas dataframes, and processed to prepare it for analysis. The processing steps include:
- filtering out stays that are less than 16 hours or greater than 128
- replace age values > 90 with 90
- standardize the continuous variables
- create the synthetic hazard function according to the formula in the paper:
  $$h(t) = H_o exp(-\lambda t) * exp(\theta A) * exp(\sum_{j=1}^{4}\beta_j Z_j)* exp(log(1.02)tZ_*) * exp(\sum_{j=1}^{4}\gamma_j g(V_j^{(t)}))$$
  where:
    - $H_o$ is the baseline hazard function
    - \lambda is the decay rate of the hazard function (risk decreases with time)
    - A is the binary treatment indicator, and $\theta$ is the treatment effect (treatment decreases risk)
    - $Z_j$ are the four static variables, and $\beta_j$ are the coefficients for the static variables
    - $Z_*$ is the indicator for severe illness, and $tZ_*$ is the interaction term between $Z_*$ and time
    - $V_j^{(t)}$ are the four time-varying variables (hematocrit, hemoglobin, platelets, mean blood pressure), and $\gamma_j$ are the coefficients for the time-varying variables
  - calculate the survival probabilites for each patient at each hour
  - using the survival probabilities, calculate the survival status for each patient at each hour
  - calculate the average treatment effect (ATE)

The output of this class is the csv file which contains the processed and synthetic data, and a series of numpy arrays for use by the Mimic3Dataset class.

In [ ]:
class Mimic3Pipeline():
    def __init__(
        self, work_dir, length_range=(16,128), min_code_count=100, n_vitals=25, seed=28
        ):
        self.work_dir = work_dir # passed in as parameter 
        self.input = pd.HDFStore(work_dir + "/data/all_hourly_data.h5")
        Path(f"{work_dir}/data/preprocessed_{seed}").mkdir(parents=True, exist_ok=True)
        self.outpath = f"data/preprocessed_{seed}"
        # minimum and maximum length of stay in hours
        self.min_length = length_range[0] # 16
        self.max_length = length_range[1] # 128
        self.min_code_counts = min_code_count # 100
        self.n_vitals = n_vitals # 25
        self.stay_lengths = None
        # create a dictionary to store arrays of data
        self.arrays = {}
        # baseline hazard
        self.H0 = 0.001
        # rate of hazard decay
        self.labda = 0.25
        self.seed = seed # 28
        np.random.seed(seed)
        # static coefficients 
        # from samples of a uniform distribution (every value equally likely to occur)
        self.beta = np.random.uniform(0.7, 1.2, size=4)
        # dynamic coefficients
        self.gamma = np.random.uniform(0.1, 0.3, size=4)
        # treatment effect on hazards
        self.alpha = -0.5

    def run(self):
        log.info("Beginning pipeline")
        # build index of patients
        # create a pandas df of interventions from the input hdf5 file
        interventions = self.input["interventions"].reset_index()
        # for each patient in the interventions df, 
        # determine the length of their stay in hours (each column is a different hour)
        stay_lengths = interventions.groupby("subject_id").size()
        # filter out patients with stays that are too short (<16 hrs) or too long (>128 hrs)
        self.stay_lengths = stay_lengths[
            (stay_lengths >= self.min_length) & (stay_lengths <= self.max_length)
            ]
        self.stay_lengths.name="stay_length"
        self.arrays["patient_index"] = self.stay_lengths.index.to_numpy()
        self.process_patients_data()
        self.process_codes()
        self.process_vitals()
        self.arrays["hourly_index"] = self.vitals.index.get_level_values(0)
        # generate labels using the function below
        self.features = self.semisynth_features()
        # fixed interventions
        self.features["treated"] = 1
        self.features["control"] = 0
        df_sim = self.simulate_treatment(self.features.copy())
        self.extract_treatment(df_sim)
        df_sim =  self.simulate_outcomes(df_sim)
        self.arrays["survival"] = df_sim["corrected_survival"].to_numpy()
        self.arrays["hazards"] = df_sim["hazard"].to_numpy()
        self.summary_statistics(df_sim)
        # save data to csv file
        log.info("Writing data")
        for key, arr in self.arrays.items():
            fname = f"{self.work_dir}/{self.outpath}/{key}.npy"
            np.save(fname, arr)
        df_sim.to_csv(f"{self.work_dir}/{self.outpath}/df_sim{self.seed}.csv")
        df_sim.to_csv(self.work_dir + f"/data/mimic3_df_{self.seed}.csv")
        log.info("Pipeline completed")

        
    def extract_treatment(self, simulated):
        # groupby col "subject_id" check if values in column "A" are True.
        # casts the resulting True/False into 1 (True) and 0 (False) using the astype() method.
        treatment = simulated.groupby("subject_id")["A"].any().astype(int)
        # This line adds the array "treatment" to the "arrays" dictionary with key "treatment".
        # to_numpy() method to obtain the Numpy array from the pandas Series object.
        self.arrays["treatment"] = treatment.to_numpy()


    def process_patients_data(self):
        # create a pandas df of patients from the input hdf5 file
        demog = self.input["patients"]
        # select gender and age columns
        demog = demog[["gender", "age"]]
        d = {"F":0, "M":1}
        # set gender to 0 or 1
        demog["gender"] = demog["gender"].apply(lambda x: d.get(x)).astype(int)
        # replace values greater than 90 with 90
        demog["age"] = demog["age"].clip(upper=90)
        # standardize age with mean 0 and std 1 using z-score
        demog["age"] = (demog["age"] - demog["age"].mean()) / demog["age"].std()
        # set the index of the df (with gender and age columns) to the subject_id
        demog = demog.reset_index().set_index("subject_id")[["gender", "age"]]
        # join the stay_lengths df with the demog df
        self.demog = demog.join(self.stay_lengths, how="right")
        self.arrays["demog"] = self.demog[["gender", "age"]].to_numpy()


    def process_codes(self):
        log.info("Processing codes")
        # create a pandas df of codes (subject_id, icd9_codes) reset the index
        codes = self.input["codes"].reset_index()[["subject_id", "icd9_codes"]].drop_duplicates(["subject_id"])
        # join the stay_lengths df with the codes df
        codes = codes.set_index("subject_id").join(self.stay_lengths, how="right")
        # explode will create a new row for each code if there is a list of codes in a row
        codes = codes.explode("icd9_codes")
        code_counts = codes["icd9_codes"].value_counts()
        # filter out codes that occur less than 100 times
        code_counts = code_counts[code_counts >= self.min_code_counts]
        code_counts.name = "count"
        code_counts = code_counts.to_frame()
        # merge the code_counts df with the codes df
        codes = codes.merge(code_counts, left_on="icd9_codes", right_index=True, how="left")
        codes["icd9_codes"] = codes["icd9_codes"].mask(codes["count"].isna())
        codes["icd9_codes"] = codes["icd9_codes"].fillna("unk")
        self.codes = codes
        self.arrays["code_index"] = codes.index.to_numpy()

        # np.unique() returns the unique values in the array and the indices of the unique values
        # "code_lookup" is a list of unique codes, "codes" are the indices of the codes in "code_lookup"
        self.arrays["code_lookup"], self.arrays["codes"] = np.unique(
            codes["icd9_codes"], return_inverse=True
            )
    

    def process_vitals(self):
        log.info("Processing vitals")
        # create a pandas df of vitals, dropping the hadm_id and icustay_id columns
        vitals = self.input["vitals_labs_mean"].droplevel(['hadm_id', 'icustay_id'])
        # reset the index
        vitals.columns = vitals.columns.get_level_values(0)
        # create a list of the top n_vitals (25) most common vitals
        vitals_list = vitals.notna().sum(0).sort_values(ascending=False).head(self.n_vitals).index
        vitals = vitals[vitals_list]
        # fill missing values with the previous value
        vitals = vitals.fillna(method="ffill")
        vitals = vitals.fillna(method="bfill")
        mean = np.mean(vitals, axis=0)
        std = np.std(vitals, axis=0)
        # standardize vitals with mean 0 and std 1 using z-score
        vitals = (vitals - mean) / std
        self.vitals = vitals.join(self.stay_lengths, how="right").drop(columns = "stay_length")
        self.arrays["vitals"] = self.vitals.to_numpy()

    def simulate_outcomes(self, df, treatment_col="A"):
        t = df.index.get_level_values(1)

        ###### Synthetic Hazard Function ######

        ## baseline hazard ##
        # The Lindy effect
        # baseline hazard = 0.001 * exp(- 0.25 * t) # using default values, t in hours
        df["baseline_hazard"] = self.H0 * np.exp(- self.labda * t)
        
        ## apply treatment effect ##
        # column can be "A", "control" (all zero), or "treat" (all one)
        df["hazard"] = df["baseline_hazard"] * np.exp(self.alpha * df[treatment_col])

        ## apply static variables ##
        # beta is from a uniform(0.7, 1.2) distribution
        X = df[["gender", "hypertension", "coronary_ath", "atrial_fib"]]
        df["hazard"] *= np.exp((X * self.beta).sum(1))

        ## apply temporal interaction ##
        # "criticl" (Z) is an indicator for severely ill patients, those with 2 or more 
        # of the following conditions: hypertension, coronary artery disease, atrial fibrillation
        # if the patient is severely ill, the hazard increases by 2% per hour,
        # thus violating the proportional hazards assumption
        df["critical"] = (df[["hypertension", "coronary_ath", "atrial_fib"]].sum(1) > 1).astype(int)
        df["hazard"] *= np.exp(np.log(1.02) * t * df["critical"])

        ## time-varying variables ##
        V = df[["hematocrit", "hemoglobin", "platelets", "mean blood pressure"]]
        # note, pandas.where:
        # if V is < 0, keep the original value, otherwise set to 0, then square the result
        V = V.where(V < 0, 0)**2
        V = V.clip(upper=3)
        df["hazard"] *= np.exp((V * self.gamma).sum(1))
        
        # stabilize hazards and convert to survival probs
        # constrain the hazard to be between 1e-8 and 0.1
        df["hazard"] = df["hazard"].clip(lower = 1e-8, upper=0.1)
        # survival probability at single time step = 1 - hazard
        df["q"] = 1 - df["hazard"]
        # survival probability at all time steps = cumulative product of survival probabilities
        df["survival_prob"] = df.groupby("subject_id")["q"].cumprod()
        # add jittering to survival probabilities
        np.random.seed(self.seed)
        # eps is a random number from a normal distribution with mean 0 and std 0.5
        # to add Gaussian noise to the logit of the survival probability
        eps = np.random.normal(loc=0, scale=0.5, size=df["survival_prob"].shape)
        df["survival_prob"] = expit(logit(df["survival_prob"]) + eps)
        # use the patient's survival probability to generate a bernoulli random variable, 
        # the first step that results in a 0 is the patient's survival time
        df["survives"] = np.random.binomial(1, df["survival_prob"])
        return self.corrected_survival_labels(df)


    def simulate_treatment(self, df):
        ## randomize treatment assignment using propensity scores ##
        # generate propensity scores
        df_flat = df.groupby(level=0).head(1)
        df_flat["critical"] = (
                df_flat[["hypertension", "coronary_ath", "atrial_fib"]].sum(1) > 1
            ).astype(int).to_numpy()
        df_flat["propensity"] = df_flat["critical"] * 0.8 + (1 - df_flat["critical"]) * 0.2
        np.random.seed(self.seed)
        # randomly assign treatment
        df_flat["A"] = np.random.binomial(1, df_flat["propensity"])
        df = df.join(df_flat["A"], how="left")
        df["A"].fillna(method="ffill", inplace=True)
        return df



    def semisynth_features(self):
        df = self.demog[["gender", "stay_length"]]
        # standardize stay length
        df["stay_length"] = (df["stay_length"] - df["stay_length"].mean()) / df["stay_length"].std()
        conf_codes = self.codes.copy()
        # indicators for hypertension, coronary artery disease, and atrial fibrillation
        conf_codes["hypertension"] = (conf_codes["icd9_codes"] == "4019")
        conf_codes["coronary_ath"] = (conf_codes["icd9_codes"] == "41401")
        conf_codes["atrial_fib"] = (conf_codes["icd9_codes"] == "42731")
        conf_codes = conf_codes.groupby(conf_codes.index)[["hypertension", "coronary_ath", "atrial_fib"]].any().astype(int)
        conf_vitals = self.vitals[["hematocrit", "hemoglobin", "platelets", "mean blood pressure"]]
        return df.join(conf_codes).join(conf_vitals)

    @staticmethod
    def corrected_survival_labels(df):
        # identify timestep at which first failure occurs (if applicable)
        first_failure = df.reset_index(level="hours_in")
        first_failure = first_failure[first_failure["survives"] == 0].groupby(level=0).first()
        first_failure = first_failure.set_index("hours_in", append=True)
        first_failure["first_failure"] = True
        first_failure = first_failure["first_failure"]
        # label censored patients
        censored = df.reset_index(level="hours_in")
        censored = (censored["survives"] == 1).groupby(level=0).all()
        censored.name = "censored"
        # combine
        df_sim = df.join(first_failure, how="left")
        df_sim = df_sim.reset_index(level="hours_in").join(censored, how="left").\
            set_index("hours_in", append=True)
        # get corrected survival labels: 1 until first failure, then zero
        df_sim["corrected_survival"] = df_sim["first_failure"]
        df_sim["corrected_survival"] = df_sim.groupby(level=0)["corrected_survival"].bfill()
        df_sim["corrected_survival"] = df_sim["corrected_survival"].fillna(False)
        df_sim["corrected_survival"] = (df_sim["corrected_survival"] | df_sim["censored"]).astype(int)
        df_sim["corrected_survival"] = df_sim["corrected_survival"].mask(df_sim["first_failure"].fillna(False), 0)
        return df_sim

    def summary_statistics(self, df_sim):
        n = df_sim.reset_index()["subject_id"].nunique()
        c = df_sim[df_sim["first_failure"] == True].shape[0]
        tau = 16
        log.info(f"{n:,} total patients")
        log.info(f"{n - c:,} censored ({100*(n - c)/n:.2f} %)")
        lifetimes = df_sim.groupby(level=0)["corrected_survival"].sum().to_numpy()
        treated_ix = df_sim.groupby(level=0)["A"].any()
        log.info(f"Mean time to censoring or failure: {np.mean(lifetimes):.2f} hours")
        rst = self.rmst(df_sim, tau)
        log.info(f"Mean restricted survival time: {np.mean(rst):.2f} hours, tau = {tau}")
        unadj_ate = rst[treated_ix].mean() - rst[~treated_ix].mean()
        log.info(f"Observed treatment effect: {unadj_ate:.2f} hours")
        # calculate true ATE
        df_treated = self.simulate_outcomes(self.features, "treated")
        rmst_treated = np.mean(self.rmst(df_treated, tau))
        df_control = self.simulate_outcomes(self.features, "control")
        rmst_control = np.mean(self.rmst(df_control, tau))
        log.info(f"True treatment effect: {rmst_treated - rmst_control:.2f} hours")

    @staticmethod
    def rmst(df, tau):
        restr = df.groupby(level=0)["corrected_survival"].head(tau)
        rst = restr.groupby(level=0).sum()
        return rst.to_numpy()


### Run the preprocessing pipeline

In [ ]:
# Import the os module
import os

# Get the current working directory
cwd = os.getcwd()

# Print the current working directory
print("Current working directory: {0}".format(cwd))
# preprocess_seed
preprocess_seed = 30
pipeline = Mimic3Pipeline(work_dir=cwd, seed=preprocess_seed)
pipeline.run()